In [11]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
import qiskit
import ipyparallel as ipp
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.providers.aer import AerSimulator
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit_nature.mappers.second_quantization import JordanWignerMapper,ParityMapper
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit.quantum_info import Pauli
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from qiskit.circuit.random import random_circuit
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator,Z2Symmetries
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy

In [12]:
def countYgates(pauli_label):
    countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
    return countYgates
def egBandHamiltonianPartition(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_0=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1_1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                line=line.split()
                i,j=map(int,line[:2])
                val=float(line[2])
                if(i==j):
                    eg_h1_0[i,j]=eg_h1_0[i+num_sites,j+num_sites]=val
                else:
                    eg_h1_1[i,j]=eg_h1_1[i+num_sites,j+num_sites]=val
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1_0=eg_h1_0+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2_0=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        eg_h2_1=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            if((i==j) and (k==l)):
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k,l] = eg_h2_0[i,j,k+num_sites,l+num_sites] \
                = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
            else:
                eg_h2_0[i,j,k,l] = eg_h2_0[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
                eg_h2_1[i+num_sites,j+num_sites,k,l] = eg_h2_1[i,j,k+num_sites,l+num_sites] = 0.5*val
    return eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1
def Energy(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,Aer.get_backend('statevector_simulator'),shots=1024).result().get_statevector()
    E=(state.conj()@Hmat@state).real
    return E

In [13]:
def commutator(A,B):
        B2=WeightedPauliOperator([[-B.paulis[0][0],B.paulis[0][1]]])
        return A.multiply(B).add(B2.multiply(A))
def commutatorsForGradient(Op):
    Op_conj=WeightedPauliOperator([[-1j,Op.paulis[0][1]]])
    M=Op.multiply(qubitH).add(qubitH.multiply(Op_conj))
    Mmat=op_converter.to_matrix_operator(M).matrix
    return Mmat
def commutatorPool(qubitH,stripZs=False):
    def stripZ(label):
        label=numpy.array(list(label))
        label[label=='Z']='I'
        label=''.join(list(label))
        return label
    #construct commutator pool from the Hamiltonian
    pool_H=[WeightedPauliOperator([[1j,qubitH.paulis[i][1]]]) for i in range(len(qubitH.paulis))][1:]
    #commutator between operators
    commutator_pool=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in pool_H for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1]
    labels=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(commutator_pool))]
    unique_labels=numpy.unique(labels)
    if stripZs==True:
        new_labels=[]
        for i in range(len(unique_labels)):
            new_labels.append(stripZ(unique_labels[i]))
        unique_labels=numpy.unique(new_labels)    
    commutator_pool=[WeightedPauliOperator([[1j,Pauli(unique_labels[i])]]) for i in range(len(unique_labels))]
    return commutator_pool
def compute_gradient(op,params):
    var_form_base.push_hopping_operator(op)
    E1=Energy(params+[numpy.pi/4.])
    E2=Energy(params+[-numpy.pi/4.])
    gradient=(E1-E2)/2
    var_form_base.pop_hopping_operator()
    return gradient

In [14]:
#Adadelta with Statevector
backend=Aer.get_backend('statevector_simulator')
def Energy_Estimator(circ):
    state=execute(circ,backend,shots=1024).result().get_statevector()
    E=(state.conj()@Hmat@state).real
    return E
def AdaDelta_SV(cost,params,runs=20,tol=1e-4,num_shots_arr=[2**8,2**9,2**9],save_opt_steps=False):
    string_shots_label=','.join([str(shots) for shots in num_shots_arr])
    v=numpy.zeros((len(params)))*0.0
    delta=numpy.zeros((len(params)))*0.0
    beta=0.9
    rho=0.8
    directions= numpy.arange(len(params))
    delta=0
    conv_err=1
    old_conv_err=1
    circ=var_form_base.construct_circuit(parameters=params)
    Eold=cost(circ)
    E_arr=[]
    ind=0
    eps=numpy.array([1e-9]*len(params))#1e-5,5e-6,1e-7,5e-8
    Max=1e5
    lr0=0.015
    def Energy_Estimator(circ):
        state=execute(circ,backend,shots=1024).result().get_statevector()
        E=(state.conj()@Hmat@state).real
        return E
    def learningRateEnv(lr0,runs):
        lr_u=lr0+Max*numpy.exp(-(37/runs)*numpy.linspace(0,runs,runs+1))
        lr_d=lr0*(1-numpy.exp(-(37/runs)*numpy.linspace(0,runs,runs+1)))
        return lr_u,lr_d
    def StochGrad(cost,params,directions):
        shiftedParams1=[numpy.array(params)+numpy.array([0]*ei+[numpy.pi/4.]+[0]*(len(params)-ei-1)) for ei in directions]
        shiftedParams2=[numpy.array(params)+numpy.array([0]*ei+[-numpy.pi/4.]+[0]*(len(params)-ei-1)) for ei in directions]
        stochGrads=numpy.zeros((len(params)))
        data_arr=[]
        for i in range(len(shiftedParams1)):
            circ=var_form_base.construct_circuit(parameters=shiftedParams1[i])
            data_arr=data_arr+[circ]
        for i in range(len(shiftedParams2)):
            circ=var_form_base.construct_circuit(parameters=shiftedParams2[i])
            data_arr=data_arr+[circ]   
        results=Parallel(n_jobs=6,verbose=2)(delayed(cost)(data_arr[i]) for i in range(len(data_arr)))    
        estm1Arr=[results[i] for i in range(0,len(results)//2)]#map(cost,shiftedParams1)
        estm2Arr=[results[i] for i in range(len(results)//2,len(results))]
        stochGrads=0.5*(numpy.array(list(estm1Arr))-numpy.array(list(estm2Arr)))
        return stochGrads
    lr_u,lr_d= learningRateEnv(lr0,runs)
    flag=0
    flag1=0
    terminate=False
    while ((conv_err>tol) and (ind<runs)):
        #print("index",ind)
        old_conv_err=conv_err
        g_stoch=StochGrad(Energy_Estimator,params,directions)
        v = beta*v+(1-beta)*g_stoch*g_stoch
        lr=numpy.sqrt(delta+eps)/numpy.sqrt(v+eps)  #learning rate
        #clipping learning rates, increasing shots and restarting ada delta
#         if ((conv_err<1e-4) and (flag==0)):
#             flag=1
#             num_shots_arr=[2**18,2**17,2**18]
#             print("************shots increased**************")
#         if ((conv_err<5e-7) and (flag1==0)):
#             flag1=1
#             num_shots_arr=[2**20,2**19,2**20]
#             print("************shots increased**************")    
        for i in range(len(lr)):
            if lr[i]>lr_u[ind]:
                eps[i]=lr_u[ind]*lr_u[ind]*v[i]
                lr[i]=lr_u[ind]
                delta[i]=0
            elif lr[i]<lr_d[ind]:
                eps[i]=lr_d[ind]*lr_d[ind]*v[i]
                lr[i]=lr_d[ind]
                delta[i]=0
            else:
                continue
        g=lr*g_stoch
        params=params-g
        params=params.real
        delta= beta*delta+(1-beta)*g*g
        E=Energy(params)
        E_arr.append(E)
        print("learning rate",lr)#,Energy(params))
        if save_opt_steps==True:
            with open('OptStepsWithSV_parityU=7.txt','a') as f:
                print('['+','.join([str(params[i]) for i in range(len(params))])+']'+'#'+str(E),file=f)
        ind=ind+1 
        conv_err=rho*conv_err+(1-rho)*numpy.abs(Eold-E)
        #print("error",conv_err)
        Eold=E
    res={'x':params,'fun':E}    
    return res

In [15]:
#preparing commutator pool
U=7
h1_D,h1_X,h2_D,h2_X=egBandHamiltonianPartition(U)
def qubitWeightedOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('parity')
    qubitH=Z2Symmetries.two_qubit_reduction(qubit_op, qubit_op.num_qubits//2)
    return qubitH
qubitH=qubitWeightedOp(h1_D+h1_X,h2_D+h2_X)
Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
w,v=numpy.linalg.eigh(Hmat)
Eg=w[0]
commutator_pool=commutatorPool(qubitH,stripZs=False)
#constructing matrices for gradient computation
#MatrixRepOfPoolOps=list(map(commutatorsForGradient,commutator_pool))
#print("pool sizes",len(MatrixRepOfPoolOps))
#backend=Aer.get_backend('statevector_simulator')
#preparing init s>tate
circ=QuantumCircuit(6)
circ.x(0)
circ.x(3)
var_form_base=UCCSD(8,num_particles=4, initial_state=circ,qubit_mapping='parity',two_qubit_reduction=True)
var_form_base.manage_hopping_operators()
error=1000

In [16]:
Energy([])

-18.111999999999988

In [17]:
import time

In [18]:
with open('OptStepsWithSV_parityU=7.txt','r') as f:
    lines=f.readlines()
    Labels=[]
    EnergyArr=[]
    for i in range(len(lines)):
        if lines[i][0]=='l':
            Labels.append(lines[i].split('-')[1][1:-1])
            if i!=0:
                
                EnergyArr.append(eval(lines[i-1].split('#')[1]))
    EnergyArr.append(eval(lines[-1].split('#')[1]))            
    params=eval(lines[-1])   
print(Labels)    
for i in range(len(Labels)):
    var_form_base.push_hopping_operator(WeightedPauliOperator([[1j,Pauli(Labels[i])]]))    

['ZYXIII', 'IIIXYI', 'YXZIII', 'IIIZXY', 'IIZIYX', 'IZZXYI', 'IZZZXY', 'IIZXYI', 'IYXIZZ', 'ZYIIIX', 'ZYIIZX', 'IIXXXY', 'IIXYXX', 'YXZIIZ', 'IIIIXY', 'IZXYXX', 'IIZXYI', 'YXXIIX', 'IIXIYZ', 'IZXXXY', 'IYZIZX', 'IIIIXY', 'IIXIIY', 'IXYIII', 'XYIIIZ']


In [19]:
EnergyArr

[-18.670874921554816,
 -18.72440014541459,
 -18.95632911611107,
 -18.978313789620223,
 -19.001030046394394,
 -19.026985736641446,
 -19.044749935785532,
 -19.088926507958824,
 -19.09762581350574,
 -19.10159032672683,
 -19.114708015305688,
 -19.119403665884665,
 -19.124151732199064,
 -19.129200662869223,
 -19.133014910859572,
 -19.134171074881287,
 -19.138638649254812,
 -19.14404990883495,
 -19.152126783560576,
 -19.158101264342832,
 -19.161625482293903,
 -19.175586128174096,
 -19.17933613770971,
 -19.180919866127752,
 -19.183038216441197]

In [ ]:
res=AdaDelta_SV(Energy_Estimator,params,runs=400,tol=1e-6,save_opt_steps=True)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   13.2s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:   16.1s finished


learning rate [0.241316   0.27526899 0.06787514 0.30812882 0.16282612 0.11001691
 0.97480724 0.09488049 0.02541944 0.02983468 0.02781972 0.02343527
 0.01602072 0.02738207 0.04854645 0.02929276 0.28760168 0.19099263
 0.02608261 0.02198816 0.03584953 0.97300367 0.01388951 0.65418397
 0.00657906]


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    6.2s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    9.6s finished


learning rate [0.11880061 0.31349582 0.10102668 0.44030725 0.23161676 0.16148904
 0.22116233 0.09092863 0.03221668 0.04416357 0.04041871 0.0345521
 0.0224213  0.02461104 0.06983411 0.0433037  0.18936055 0.04715988
 0.03885708 0.03242709 0.05178301 0.20946427 0.01978598 0.61803388
 0.0067464 ]


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    5.3s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:    8.3s finished


...

learning rate [0.03852775 0.03852775 0.03852775 0.03852775 0.03852775 0.03852775
 0.03852775 0.03852775 0.03852775 0.03852775 0.03852775 0.03852775
 0.02874559 0.03139447 0.03852775 0.03852775 0.03852775 0.03852775
 0.03852775 0.03852775 0.03852775 0.03852775 0.03037931 0.03852775
 0.03852775]


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    7.7s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:   13.0s finished


learning rate [0.03644906 0.03644906 0.03644906 0.03644906 0.03644906 0.03644906
 0.03644906 0.03644906 0.03644906 0.03644906 0.03644906 0.03644906
 0.02893352 0.0315013  0.03644906 0.03644906 0.03644906 0.03644906
 0.03644906 0.03644906 0.03644906 0.03644906 0.03051618 0.03644906
 0.03644906]


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    6.4s
[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:   11.7s finished


learning rate [0.03455402 0.03455402 0.03455402 0.03455402 0.03455402 0.03455402
 0.03455402 0.03455402 0.03455402 0.03455402 0.03455402 0.03455402
 0.02918852 0.03159863 0.03455402 0.03455402 0.03455402 0.03455402
 0.03455402 0.03455402 0.03455402 0.03455402 0.03071361 0.03455402
 0.03455402]


In [ ]:
#params=[]
#EnergyArr=[]
ExcOps=Labels
ti=time.time()
steps=35
print_res=False
for i in range(35):
    grads=numpy.array(Parallel(n_jobs=1,verbose=2)(delayed(compute_gradient)(op,params) for op in commutator_pool))
    indexes=numpy.argsort(abs(grads))[::-1][:5]
    #Labels=[commutator_pool[indexes[i]].paulis[0][1].to_label() for i in range(len(indexes))]
    print("3 highest grads",grads[indexes[0]],grads[indexes[1]],grads[indexes[2]])
    print("3 highest grads",commutator_pool[indexes[0]].paulis[0][1].to_label(),commutator_pool[indexes[1]].paulis[0][1].to_label(),commutator_pool[indexes[2]].paulis[0][1].to_label())
    if len(ExcOps)!=0:
        if (commutator_pool[indexes[0]].paulis[0][1].to_label()!=ExcOps[-1]):
            PauliOp=commutator_pool[indexes[0]]
        else:
            PauliOp=commutator_pool[indexes[1]]
    else:
        PauliOp=commutator_pool[indexes[0]]
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    print("chosen Op",ExcOps[-1])
    with open('OptStepsWithSV_parityU=7.txt','a') as f:
        print("label-",ExcOps[-1],file=f)
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    res=AdaDelta_SV(Energy_Estimator,params,runs=400,tol=1e-6,save_opt_steps=True)#SMO_qasm(hamiltonian_estimation_in_doubly_decomposed_form_qasm,params,runs=40,tol=1e-6,save_opt_steps=True)    #res=scipy.optimize.minimize(Energy_Estimator,params,method='BFGS',options={'ftol':1e-7})
    params,E=list(res['x']),res['fun']
    print("num_params",var_form_base.num_parameters)
    print("Energy",E)
    EnergyArr.append(E)
    print("time elapsed",time.time()-ti)    
    error=EnergyArr[-1]-Eg

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.0s remaining:    0.0s


3 highest grads 0.01681735523232497 0.01469790862321041 -0.013367110030515938
3 highest grads IZYIIX IIYIIX IIYIZX
chosen Op IZYIIX


[Parallel(n_jobs=1)]: Done  56 out of  56 | elapsed:  2.8min finished
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   19.3s
[Parallel(n_jobs=6)]: Done  52 out of  52 | elapsed:   27.3s finished


learning rate [0.2839102  0.78347183 0.3924191  0.87319558 0.2894299  0.37337579
 0.16037208 0.07973128 0.14286373 0.27750655 0.36578042 0.24103398
 0.08216756 0.02356182 0.99904714 0.80495313 0.09323531 0.0665529
 0.53215297 0.31866098 0.11281956 0.17363481 0.10246879 0.14491313
 0.0272884  0.00594613]


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    6.6s
[Parallel(n_jobs=6)]: Done  52 out of  52 | elapsed:   11.6s finished


...

learning rate [0.12707243 0.33062978 0.12919179 0.38543855 0.28993458 0.1959822
 0.19650655 0.15183529 0.06928753 0.05588561 0.05313625 0.04880807
 0.04624737 0.02501807 0.08939059 0.05723559 0.21937807 0.05048576
 0.04855487 0.04355027 0.06143633 0.16549029 0.03666208 0.34074459
 0.03491808 0.02697498]


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    5.3s
[Parallel(n_jobs=6)]: Done  52 out of  52 | elapsed:   10.5s finished


learning rate [0.12582085 0.33613649 0.13020437 0.4002794  0.27858338 0.18656774
 0.20101109 0.13898369 0.06976391 0.05720072 0.05290749 0.04709547
 0.04733104 0.02638619 0.08258698 0.05639219 0.21741495 0.05030117
 0.04711997 0.04119659 0.06068665 0.17943143 0.03541826 0.3543915
 0.03473541 0.02985317]


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    5.4s
[Parallel(n_jobs=6)]: Done  52 out of  52 | elapsed:    9.3s finished


learning rate [0.12712438 0.3314737  0.12918624 0.38585798 0.28935124 0.19535962
 0.19665176 0.15105579 0.06925101 0.05581004 0.05305691 0.04869787
 0.04622524 0.02507818 0.08894096 0.05715961 0.2221534  0.05041494
 0.04845406 0.04338877 0.06137529 0.16574216 0.03661118 0.34699307
 0.03509312 0.02748296]


In [ ]:
indexes=numpy.argsort(abs(grads))[::-1][:5]
label=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(indexes))]

In [ ]:
label

In [ ]:
grads